### * RNN 주요 레이어 종류
#### (1) SimpleRNN :가장 간단한 형태의 RNN레이어, 활성화 함수로 tanh가 사용됨(tanh: -1 ~ 1 사이의 값을 반환)
#### (2) LSTM(Long short Term Memory) : 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제), LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함, 활성화 함수로 tanh외에 sigmoid가 사용됨
#### (3) GRU(Gated Recurent Unit) : 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수함

<<참고 : https://colah.github.io/posts/2015-08-Understanding-LSTMs/ >>
https://link.springer.com/article/10.1007/s11063-009-9096-2

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [3]:
# Sequence data
x = np.array([[0,1,2,3],
              [1,2,3,4],
              [2,3,4,5],
              [3,4,5,6],
              [4,5,6,7],
              [5,6,7,8]],dtype=np.float32)

x_data = tf.reshape(x, (-1,4,1))  # (6,4,1)

y_data = np.array([4,5,6,7,8,9],dtype=np.float32)

print(x_data.shape,y_data.shape)
# print(type(x_data),type(y_data))
x_data

(6, 4, 1) (6,)


<tf.Tensor: shape=(6, 4, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [2.],
        [3.]],

       [[1.],
        [2.],
        [3.],
        [4.]],

       [[2.],
        [3.],
        [4.],
        [5.]],

       [[3.],
        [4.],
        [5.],
        [6.]],

       [[4.],
        [5.],
        [6.],
        [7.]],

       [[5.],
        [6.],
        [7.],
        [8.]]], dtype=float32)>

### [1] SimpleRNN
#### 가장 간단한 형태의 RNN 

In [4]:
#Q1)  SimpleRNN모델 구현하자.  
#X :  (6,4,1)   reshape한 데이터
# Wx : (1,300 )   ,  입력 가중치   (입력차원, 뉴런) = 파라미터 개수 = 1*300=300개
# Wh (300,300) ,  숨겨진 상태 가중치     (유닛개수, 숨겨진 상태) 그래서 파라미터 수는 300*300=90,000
# b (300) , param 90600      = Wx 300 + Wh 90,000 + b 300 = 90,600
# OUt (None, 4, 300)  
model = Sequential([
    SimpleRNN(units=300, return_sequences=True, input_shape=[4,1]),
    
#X :  (None, 4, 300)   
# Wx : (300,300 )   , Wh (300,300) , b (300) , param 180300   , OUt (None,300)  
    SimpleRNN(units=300),
    Dense(1)      # 출력 차원이 1, 가중치 형태 = (300, 1) = 300 + 1 = 301
])

model.compile(optimizer='adam',loss='mse')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 300)            90600     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 300)               180300    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 271201 (1.03 MB)
Trainable params: 271201 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_data, y_data, epochs=100, verbose=1)
print(model.predict(x_data))

In [ ]:
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

In [ ]:
model.evaluate(x_data, y_data)

### [2] LSTM(Long short Term Memory)
#### 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제)
#### LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함

In [ ]:
#Q2) RNN 순환 신경망 구현  : LSTM
model = Sequential([   
 #X :  (6,4,1) 
# Wx : (1,4*300 )   , Wh (300,4*300) , b (4*300) , param  4*90600      , OUt (None, 4, 300)
    LSTM(units=300, return_sequences=True, input_shape=[4,1]),  
    
  #X :  None, 4, 300
# Wx : (300,4*300 )   , Wh (300,4*300) , b (4*300) , param  4*180300     , OUt   (None, 300) 
    LSTM(units=300),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

In [ ]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

In [ ]:
model.evaluate(x_data,y_data)

### [3] GRU(Gated Recurent Unit)
#### 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수

In [ ]:
# RNN 순환 신경망 구현  : GRU
model = Sequential([
    GRU(units=300, return_sequences=True, input_shape=[4,1]),
    GRU(units=300),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

In [ ]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))